# Homework 4: Modeling Text Data

### Team Member 1:
* UNI: gc2708 
* Name: GwonJae Cho

### Team Member 2 [optional]:
* UNI:  
* Name:


You can find the data here: https://data.boston.gov/dataset/vision-zero-entry

# Task1 - Data Cleaning  [10 points]

Load the data, visualize the class distribution. Clean up the target labels. Some categories have been arbitrarily split and need to be consolidated. 

In [1]:
# Add your code for task 1 here. You may use multiple cells. 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

% matplotlib inline
plt.rcParams["figure.dpi"] = 100
np.set_printoptions(precision=3, suppress=True)
plt.style.use(['fivethirtyeight'])

df = pd.read_csv('Vision_Zero_Entry.csv')
df = df[["REQUESTTYPE","COMMENTS"]]
df.dropna(axis=0, inplace=True)
df.isnull().any()

df.drop_duplicates(inplace=True, keep='last')

# Remove and consolidate duplicates
df["REQUESTTYPE"].replace("there are no bike facilities or they need maintenance","bike facilities don't exist or need improvement",inplace=True)
df["REQUESTTYPE"].replace('" src="images/14 - Other issue.png"></span>&nbsp;of something that is not listed here', 'of something that is not listed here',inplace=True)
df["REQUESTTYPE"].replace(['" src="images/02 - Wait is too long.png"></span>&nbsp;the wait for the "Walk" signal is too long', 
                           'people have to wait too long for the "Walk" signal'],
                          'the wait for the "Walk" signal is too long',inplace=True)
df["REQUESTTYPE"].replace(['" src="images/11 - Sidewalk issue.png"></span>&nbsp;sidewalks/ramps don\'t exist or need improvement', 
                           'there are no sidewalks or they need maintenance'],
                          "sidewalks/ramps don't exist or need improvement",inplace=True)
df["REQUESTTYPE"].replace(["""\" src=\"images/06 - Speeding.png"></span>&nbsp;people speed"""],
                          "people speed",inplace=True)

df["REQUESTTYPE"].replace("people have to cross too many lanes / too far",
                          "it's too far / too many lanes to cross",inplace=True)

df["REQUESTTYPE"].replace(['" src="images/01 - Not enough time to cross.png"></span>&nbsp;there\'s not enough time to cross the street', 
                           'people are not given enough time to cross the street'],
                          "there's not enough time to cross the street",inplace=True)

df["REQUESTTYPE"].replace(['" src="images/12 - Bike facility issue.png"></span>&nbsp;the roadway surface needs improvement', 
                           'the roadway surface needs maintenance'],
                          "the roadway surface needs improvement",inplace=True)
df["REQUESTTYPE"].replace(['" src="images/10 - Hard to see.png"></span>&nbsp;it’s hard to see / low visibility', 
                           'it’s hard for people to see each other'],
                          "it’s hard to see / low visibility",inplace=True)


target = df[["REQUESTTYPE"]]
df = df[["COMMENTS"]]
target_c = target

In [3]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

target = le.fit_transform(target)

from sklearn.model_selection import train_test_split
X_train_sub, X_test_val, y_train_sub, y_test_val = train_test_split(df, target, random_state=0)

[0 3 5 ..., 9 6 2]


# Task2 - Model 1 [10 points]

Run a baseline multi-class classification model using a bag-of-word approach, report macro f1-score (should be above .5) and visualize the confusion matrix. Can you interpret the mistakes made by the model? 

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

#Vanilla bag of words
vect = CountVectorizer()
X_train = vect.fit_transform(X_train_sub["COMMENTS"])
X_test = vect.transform(X_test_val["COMMENTS"])

from sklearn.linear_model import LogisticRegressionCV

lr = LogisticRegressionCV().fit(X_train, y_train_sub)

from sklearn.metrics import f1_score
pred = lr.predict(X_test)
assert(f1_score(y_test_val, pred , average = 'macro') > 0.35)


0.504366642269


# Task3 - Model 2 [30 points]

Improve the model using more complex text features, including n-grams, character n-grams and possibly domain-specific features.

In [6]:
# Bag of words with ngram = (1,3)
cv = CountVectorizer(ngram_range=(1,3),min_df=3,stop_words='english')
X_train = cv.fit_transform(X_train_sub["COMMENTS"])
X_test = cv.transform(X_test_val["COMMENTS"])

lr = LogisticRegressionCV().fit(X_train, y_train_sub)

pred = lr.predict(X_test)
assert(f1_score(y_test_val, pred , average = 'macro') > 0.45)

[2 3 8 ..., 9 3 8]
[ 2 10  5 ...,  9  3  5]
0.515836722743


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Used character word boundary
cv = CountVectorizer(ngram_range=(2,3),min_df=3, analyzer='char_wb')
X_train = cv.fit_transform(X_train_sub["COMMENTS"])
X_test = cv.transform(X_test_val["COMMENTS"])

lr = LogisticRegressionCV().fit(X_train, y_train_sub)

pred = lr.predict(X_test)
assert(f1_score(y_test_val, pred , average = 'macro') > 0.4)

[2 0 5 ..., 8 3 8]
[ 2 10  5 ...,  9  3  5]
0.502563559343


# Task4 - Visualize Results [10 points]

Visualize results of the tuned model (classification results, confusion matrix, important features, example mistakes).

# Task5 - Clustering [10 points]

Apply LDA, NMF and K-Means to the whole dataset. Can you find clusters or topics that match well with some of the ground truth labels? Use ARI to compare the methods and visualize topics and clusters.

In [11]:
# from sklearn.decomposition import LatentDirichletAllocation
# import mglearn
# from mglearn.tools import print_topics

# # LDA to extract topics
# vect = CountVectorizer(ngram_range=(1,2), stop_words='english')
# X = vect.fit_transform(df["COMMENTS"])

# lda = LatentDirichletAllocation(n_topics=12, learning_method="batch", max_iter=25, random_state=0)

# document_topics = lda.fit_transform(X)
    
# sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# feature_names = np.array(vect.get_feature_names())

# mglearn.tools.print_topics(topics=range(13), feature_names=feature_names,
# sorting=sorting, topics_per_chunk=5, n_words=20)

topic 0                 topic 1                 topic 2                 topic 3                 topic 4                 
-----------------      -----------------      -----------------      -----------------      -----------------      
street                        cars                          light                         left                          stop                          
cars                          bike                          red                           traffic                       st                            
people                        road                          run                           turn                          traffic                       
pedestrians                   sidewalk                      turn                          going                         street                        
ave                           crosswalk                     cars                          lane                          people                        
st      

In [12]:
# from sklearn.decomposition import NMF
# from sklearn.feature_extraction.text import TfidfVectorizer

# # NMF for extracting topics
# vect = TfidfVectorizer(ngram_range=(1,3), min_df=3,stop_words='english')
# X = vect.fit_transform(df["COMMENTS"])
# nmf = NMF(n_components=15, verbose=10, tol=0.01)
# nmf.fit(X)
# sorting = np.argsort(nmf.components_, axis=1)[:, ::-1]
# feature_names = np.array(vect.get_feature_names())

# # Visualize the NMF result
# mglearn.tools.print_topics(topics=range(13), feature_names=feature_names,
# sorting=sorting, topics_per_chunk=5, n_words=20)

violation: 1.0
violation: 0.4510477041243306
violation: 0.27204885362963893
violation: 0.14776041303408743
violation: 0.094007723440683
violation: 0.07096002859540299
violation: 0.059976253410998694
violation: 0.052858228949005145
violation: 0.046605264460541695
violation: 0.04123467803110079
violation: 0.03725625740304253
violation: 0.035018522253607645
violation: 0.035283110624897585
violation: 0.03780478529215002
violation: 0.04172468519395488
violation: 0.04558489674116509
violation: 0.04722673112807802
violation: 0.045714042372578256
violation: 0.04022042014822228
violation: 0.0319667764072917
violation: 0.022985822666343688
violation: 0.01562713949535939
violation: 0.010624805067358349
violation: 0.007589018299919157
Converged at iteration 24
topic 0                 topic 1                 topic 2                 topic 3                 topic 4                 
-----------------      -----------------      -----------------      -----------------      -----------------      
ligh

In [13]:
# from sklearn.cluster import KMeans
# from sklearn import metrics

# km = KMeans(n_clusters=13)
# X = vect.fit_transform(df["COMMENTS"])
# %time km.fit(X)

# clusters = km.labels_.tolist()
# print("Adjusted Rand-Index: %.3f"
#       % metrics.adjusted_rand_score(target, clusters))

CPU times: user 42.5 s, sys: 640 ms, total: 43.2 s
Wall time: 21.8 s
Adjusted Rand-Index: 0.089


# Task6 - Model 3 [30 points]

Improve the class definition for REQUESTTYPE by using the results of the clustering and results of the previous classification model. Re-assign labels using either the results of clustering or using keywords that you found during data exploration. The labels must be semantically meaningful.
The data has a large “other” category. Apply the topic modeling and clustering techniques to this subset of the data to find possible splits of this class.
Report accuracy using macro average f1 score (should be above .53) 


In [14]:
# import mglearn
# from mglearn import tools

# vect = CountVectorizer(ngram_range=(3,5), min_df=2, stop_words='english')
# X = vect.fit_transform(df["COMMENTS"])


# lda = LatentDirichletAllocation(n_topics=5, learning_method="batch", max_iter=25, random_state=0)

# document_topics = lda.fit_transform(X[np.where(target==3)])
# sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# feature_names = np.array(vect.get_feature_names())

# mglearn.tools.print_topics(topics=range(5), feature_names=feature_names,
# sorting=sorting, topics_per_chunk=3, n_words=20)


(array([0, 1, 2, 3, 4]), array([705, 189, 141, 178, 187]))
topic 0                 topic 1                 topic 2                 
-----------------      -----------------      -----------------      
drive wrong way               going wrong way               pedestrian crossing light     
wrong way way                 right turn lane               slow pedestrian crossing      
wrong way way street          chestnut hill ave             cross intersection pedestrian 
way way street                bicyclists ride sidewalk      cross intersection pedestrian light
left turn lane                cars turning left             feel safer crossing mid block 
people drive wrong            hyde park ave                 feel safer crossing mid       
people drive wrong way        middle lane turn              feel safer crossing           
hyde park ave                 people right lane             mid block attempt             
drive wrong way way           left lane straight            mid b

In [16]:
# # with the above clustering, we can see going wrong way can be one new label which semantically meaningful
# target_c[df['COMMENTS'].str.contains("wrong way")] = "going wrong way"

# le=LabelEncoder()
# target_c = le.fit_transform(target_c)
# X_train_sub, X_test_val, y_train_sub, y_test_val = train_test_split(df, target_c, random_state=0)

# cv = CountVectorizer(ngram_range=(1,3),min_df=3,stop_words='english')
# X_train = cv.fit_transform(X_train_sub["COMMENTS"])
# X_test = cv.transform(X_test_val["COMMENTS"])

# lr = LogisticRegressionCV().fit(X_train, y_train_sub)


# pred = lr.predict(X_test)
# assert(f1_score(y_test_val, pred , average = 'macro') > 0.53)

[ 3  4  9 ..., 10  4  9]
[ 3 11  6 ..., 10  4  6]
0.548147387817


# Extra Credit [Up to +20 points]

Use a word embedding representation like word2vec for step 3 and or step 6. 

In [17]:
# Add your code for extra credit here. You may use multiple cells. 

